For this project you'll dig into a large amount of text and apply most of what you've covered in this unit and in the course so far.

First, pick a set of texts. This can be either a series of novels, chapters, or articles. Anything you'd like. It just has to have multiple entries of varying characteristics. At least 100 should be good. There should also be at least 10 different authors, but try to keep the texts related (either all on the same topic of from the same branch of literature - something to make classification a bit more difficult than obviously different subjects).

This capstone can be an extension of your NLP challenge if you wish to use the same corpus. If you found problems with that data set that limited your analysis, however, it may be worth using what you learned to choose a new corpus. Reserve 25% of your corpus as a test set.

The first technique is to create a series of clusters. Try several techniques and pick the one you think best represents your data. Make sure there is a narrative and reasoning around why you have chosen the given clusters. Are authors consistently grouped into the same cluster?

Next, perform some unsupervised feature generation and selection using the techniques covered in this unit and elsewhere in the course. Using those features then build models to attempt to classify your texts by author. Try different permutations of unsupervised and supervised techniques to see which combinations have the best performance.

Lastly return to your holdout group. Does your clustering on those members perform as you'd expect? Have your clusters remained stable or changed dramatically? What about your model? Is its performance consistent?

If there is a divergence in the relative stability of your model and your clusters, delve into why.

Your end result should be a write up of how clustering and modeling compare for classifying your texts. What are the advantages of each? Why would you want to use one over the other? Approximately 3-5 pages is a good length for your write up, and remember to include visuals to help tell your story!

In [1]:
# Set up data science environment.
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import seaborn as sns
import math
import scipy
import spacy
import re
import warnings
from bokeh.io import output_notebook
from bokeh.layouts import column
from bokeh.models import HoverTool, CustomJS, ColumnDataSource, Slider
from bokeh.palettes import all_palettes
from bokeh.plotting import figure, show
from collections import Counter
from gensim import corpora, models
from gensim.models.ldamodel import LdaModel
from nltk.corpus import inaugural, stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.cluster import KMeans, MiniBatchKMeans, SpectralClustering
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler, normalize, Normalizer
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from typing import Dict
output_notebook()

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format
sns.set_style('white')

# Suppress annoying harmless error.
warnings.filterwarnings(
    action='ignore',
    module='scipy',
    message='internal gelsd'
)

Loading BokehJS ...

# Data Cleaning, Processing, and Language Parsing

In [2]:
# Create lists for files and presidents.
files = ["1789-Washington.txt",
         "1801-Jefferson.txt",
         "1861-Lincoln.txt",
         "1933-Roosevelt.txt",
         "1953-Eisenhower.txt",
         "1961-Kennedy.txt",
         "1981-Reagan.txt",
         "1989-Bush.txt",
         "1993-Clinton.txt",
         "2009-Obama.txt"]

presidents = ["washington",
              "jefferson",
              "lincoln",
              "fdr",
              "eisenhower",
              "kennedy",
              "reagan",
              "ghwbush",
              "clinton",
              "obama"]

# Control to make sure both lists are the same length.
assert len(files) == len(presidents)

In [3]:
# Quick function to open all files needed (and close them again).
docs = []
for file_name, president in zip(files, presidents):
    with open(f'./inaugural/{file_name}') as f:
        doc = f.read()
        docs.append((doc, president))

In [4]:
# Utility function to clean text.
def text_cleaner(text: str) -> str:
    """Function to strip all characters except letters in words."""
    
    text = re.sub(r'--', ' ', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub("[\<].*?[\>]", "", text)
    text = ' '.join(text.split())
    return text

In [5]:
# Use text_cleaner on the docs, combine them into data frame (clean_docs).
clean_docs = []
for doc, pres in docs:
    clean_doc = text_cleaner(doc)
    clean_docs.append((clean_doc, pres))

In [6]:
# Iterate through each doc and print the first 1000 characters for inspection.
for doc, pres in clean_docs:
    print(doc[:1000], pres.upper()) 
    print()

Fellow-Citizens of the Senate and of the House of Representatives: Among the vicissitudes incident to life no event could have filled me with greater anxieties than that of which the notification was transmitted by your order, and received on the th day of the present month. On the one hand, I was summoned by my Country, whose voice I can never hear but with veneration and love, from a retreat which I had chosen with the fondest predilection, and, in my flattering hopes, with an immutable decision, as the asylum of my declining years a retreat which was rendered every day more necessary as well as more dear to me by the addition of habit to inclination, and of frequent interruptions in my health to the gradual waste committed on it by time. On the other hand, the magnitude and difficulty of the trust to which the voice of my country called me, being sufficient to awaken in the wisest and most experienced of her citizens a distrustful scrutiny into his qualifications, could not but over

In [7]:
# Define nlp as spacy.
nlp = spacy.load('en')
# Create an empty list for df.
df_list = []


# Create a function to parse data.
def nlp_text(text_file: str) -> doc:
    """Function that takes a text file and tokenizes it with spacy."""
    
    
    doc = nlp(text_file)
    return doc


# Create a function to lemmatize sentences.
def sentences(doc_nlp: str, speaker: str) -> [str, str]:
    """Function that takes two strings, lemmatizes the first string and 
    returns a list with two strings.
    """
    
    
    return [[sent.lemma_, speaker] for sent in doc_nlp.sents]


# Create a function to combine groups of sentences into one data frame.
def sentences_to_df(sents):
    """Function that takes a string and returns a data frame."""

    
    return pd.DataFrame(sents)


# Calling each function.
for doc, pres in clean_docs:
    parsed = nlp_text(doc)
    sents = sentences(parsed, pres)
    df = sentences_to_df(sents)
    df_list.append(df)

In [8]:
# Combine each sentence data frame into one master data frame.
sent_df = pd.concat([*df_list])

In [9]:
# Rename columns.
sent_df.columns = ['sentence', 'President']

# Check the count of sents per President.
sent_df.President.value_counts()

ghwbush       145
lincoln       139
reagan        130
eisenhower    121
obama         113
fdr            86
clinton        82
kennedy        53
jefferson      42
washington     25
Name: President, dtype: int64

In [10]:
# Filter out pronouns from results.
sent_df['sentence'] = sent_df['sentence'].str.replace('-PRON-', '')

# Creating Features

In [11]:
# Splitting the data.
X = sent_df.sentence
y = sent_df.President
X_train_eval, X_holdout, y_train_eval, y_holdout = train_test_split(
    X, y, test_size=0.25, random_state=15)

In [12]:
# Splitting into train/eval/holdout groups.
X_train, X_eval, y_train, y_eval = train_test_split(
    X_train_eval, y_train_eval, test_size=0.25, random_state=15)

In [13]:
# Create base parameters dictionary.
base_param_dict = {'strip_accents': 'unicode',
                   'lowercase': True,
                   'stop_words': 'english',
                   'ngram_range': (1, 3),
                   'max_df': 0.5,
                   'min_df': 5,
                   'max_features': 1000}

## Bag of Words

In [14]:
# Instantiate CountVectorizer.
bow = CountVectorizer(**base_param_dict)

In [15]:
# Convert X_train, X_test into dfs of bags of words.
_bow_train = bow.fit_transform(X_train)
_bow_eval = bow.transform(X_eval)
_bow_holdout = bow.transform(X_holdout)
assert len(X_train) == _bow_train.shape[0]  # df and sparse-matrix

# Find feature names.
feature_names = bow.get_feature_names()

# Sparse matrix to data frame.
X_train_bow = pd.DataFrame(_bow_train.toarray(), columns=feature_names)
X_eval_bow = pd.DataFrame(_bow_eval.toarray(), columns=feature_names)
X_holdout_bow = pd.DataFrame(_bow_holdout.toarray(), columns=feature_names)

## Tfidf

In [16]:
# Instantiate Tfidf.
tfidf = TfidfVectorizer(**base_param_dict)

In [17]:
# Convert X_train, X_test into scipy sparse matrices of tfidf values.
_tfidf_train = tfidf.fit_transform(X_train)
_tfidf_eval = tfidf.transform(X_eval)
_tfidf_holdout = tfidf.transform(X_holdout)
assert len(X_train) == _tfidf_train.shape[0]  # df and sparse-matrix

# Find feature names.
feature_names_tfidf = tfidf.get_feature_names()

# Sparse matrix to data frames.
X_train_tfidf = pd.DataFrame(
    _tfidf_train.toarray(), columns=feature_names_tfidf)
X_eval_tfidf = pd.DataFrame(
    _tfidf_eval.toarray(), columns=feature_names_tfidf)
X_holdout_tfidf = pd.DataFrame(
    _tfidf_holdout.toarray(), columns=feature_names_tfidf)

In [18]:
# Calculate weights on training data.
weights = np.asarray(X_train_tfidf.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame(
    {'word': tfidf.get_feature_names(), 'avg_weight': weights})
print("\nTrain Weights:\n", weights_df.sort_values(
    by='avg_weight', ascending=False).head(10))

# Calculate weights on eval data.
weights = np.asarray(X_eval_tfidf.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame(
    {'word': tfidf.get_feature_names(), 'avg_weight': weights})
print("\nEval Weights:\n", weights_df.sort_values(
    by='avg_weight', ascending=False).head(10))


Train Weights:
            word  avg_weight
147      people       0.032
227       world       0.028
80   government       0.028
131      nation       0.026
206        time       0.025
108         let       0.023
79         good       0.023
120        make       0.022
111        life       0.022
81        great       0.022

Eval Weights:
            word  avg_weight
135         new       0.029
226        work       0.029
81        great       0.028
74      freedom       0.027
132    national       0.027
194      states       0.025
131      nation       0.024
80   government       0.023
147      people       0.023
158   principle       0.023


## Latent Semantic Analysis (LSA)

In [19]:
# Instantiate MinMaxScaler. Create train/eval/holdout groups for Tfidf.
scaler = MinMaxScaler()

# Tfidf
#X_train_tfidf_scaled = pd.DataFrame(
#    scaler.fit_transform(X_train_tfidf), columns=feature_names_tfidf)
#X_eval_tfidf_scaled = pd.DataFrame(
#    scaler.transform(X_eval_tfidf), columns=feature_names_tfidf)
#X_holdout_tfidf_scaled = pd.DataFrame(
#    scaler.transform(X_holdout_tfidf), columns=feature_names_tfidf)

In [20]:
# Reduce feature space to 100 features.
svd = TruncatedSVD(100)

# Make pipeline to run SVD and normalize results.
lsa_pipe = make_pipeline(svd, Normalizer())

# Fit with training data, transform test data.
X_train_lsa = lsa_pipe.fit_transform(X_train_tfidf)
X_eval_lsa = lsa_pipe.transform(X_eval_tfidf)
X_holdout_lsa = lsa_pipe.transform(X_holdout_tfidf)

# Examine variance captured in reduced feature space.
variance_explained = svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print('Percent variance captured by components:', total_variance*100)

sent_by_component = pd.DataFrame(X_train_lsa, index=X_train)

# Look at values from first 5 components.
for i in range(6):
    print('Component {}:'.format(i))
    print(sent_by_component.loc[:, i].sort_values(ascending=False)[:5])

Percent variance captured by components: 74.92686660519296
Component 0:
sentence
 government have no power except that grant  by the people .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [21]:
# Create train/eval/holdout groups for LSA.
X_train_lsa_scaled = pd.DataFrame(scaler.fit_transform(X_train_lsa))
X_eval_lsa_scaled = pd.DataFrame(scaler.transform(X_eval_lsa))
X_holdout_lsa_scaled = pd.DataFrame(scaler.transform(X_holdout_lsa))

# Clustering Models

In [22]:
# Clustering models
models = []
names = []
plot_nums = []
silhouettes = []
clust = []

for clusters in range(2, 11):
    models.append(
        (0, 'KMeans', KMeans(n_clusters=clusters,
                             init='k-means++', random_state=15)))
    models.append(
        (1, 'MiniBatch', MiniBatchKMeans(init='random',
                                         n_clusters=clusters,
                                         batch_size=500)))
    #models.append((2, 'Spectral', SpectralClustering(n_clusters=clusters)))

for plot_num, name, model in models:
    names.append(name)
    model.fit(X_train_tfidf)
    labels = model.labels_
    print(model)
    if len(set(labels)) > 1:
        ypred = model.fit_predict(X_train_tfidf)
        silhouette = metrics.silhouette_score(
            X_train_tfidf, labels, metric='euclidean')
        silhouettes.append(silhouette)
        #ax[plot_num].set_title(name)
        #plotting(plot_num, labels, ypred)
        if silhouette > 0:
            print('clusters: {}\t silhouette: {}\n'.format(
                model.n_clusters, silhouette))
            print(name, '\n', pd.crosstab(ypred, labels), '\n')

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=15, tol=0.0001, verbose=0)
clusters: 2	 silhouette: 0.02403808152620107

KMeans 
 col_0    0   1
row_0         
0      487   0
1        0  39 

MiniBatchKMeans(batch_size=500, compute_labels=True, init='random',
        init_size=None, max_iter=100, max_no_improvement=10, n_clusters=2,
        n_init=3, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=0)
clusters: 2	 silhouette: 0.02483958468795499

MiniBatch 
 col_0    0   1
row_0         
0      397  29
1       90  10 

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=15, tol=0.0001, verbose=0)
clusters: 3	 silhouette: 0.024610759668237767

KMeans 
 col_0    0   1   2
row_0             
0      395   0   0
1        0  59   0
2        0   0  72 

MiniBat

clusters: 10	 silhouette: 0.034095956926305335

MiniBatch 
 col_0    0   1   2   3   4  5   6   7   8  9
row_0                                       
0        9   0   1   3  12  0   1   3   3  1
1        4   0   0   0   0  0   0  10   2  2
2        2   0   0   1   2  0  34   0   3  1
3        2   0   0   0   4  0   0   2   1  0
4      168   6  11  14  12  1   6  16  39  8
5        3  13   0   0   0  0   0   0   1  0
6       25   0   1   0   3  0   3   2   4  0
7       22   0   1   1   1  3   8   2   1  2
8       20   0   0   0   1  0   0   0   1  0
9       10   0   0   1   2  0   2   1   8  0 



## Tfidf

In [23]:
# Re-run KMeans and extract cluster information.
model_tfidf = KMeans(n_clusters=10, random_state=15).fit(X_train_tfidf)

# Extract cluster assignments for each data point.
labels = model_tfidf.labels_

In [24]:
# Create cluster assignment for eval, holdout groups.
X_eval_tfidf_labels = model_tfidf.predict(X_eval_tfidf)
X_holdout_tfidf_labels = model_tfidf.predict(X_holdout_tfidf)

# Create a column for cluster labels.
X_eval_tfidf['clusters'] = X_eval_tfidf_labels
X_holdout_tfidf['clusters'] = X_holdout_tfidf_labels

X_train_tfidf['clusters'] = labels

In [25]:
# Aggregate by cluster.
X_train_tfidf_clusters = X_train_tfidf.groupby(
    ['clusters'], as_index=False).mean()
X_train_tfidf_clusters

,clusters,act,action,administration,america,american,american people,americans,ask,authority,...,way,willing,wish,woman,word,work,world,write,year,young
0,0,0.015,0.005,0.003,0.019,0.002,0.000,0.011,0.010,0.011,...,0.003,0.000,0.004,0.006,0.001,0.000,0.001,0.006,0.003,0.007
1,1,0.000,0.000,0.000,0.007,0.007,0.000,0.014,0.012,0.000,...,0.000,0.013,0.007,0.008,0.009,0.006,0.025,0.000,0.000,0.017
2,2,0.000,0.000,0.000,0.017,0.000,0.000,0.012,0.000,0.000,...,0.030,0.030,0.000,0.000,0.010,0.000,0.017,0.000,0.015,0.000
3,3,0.000,0.000,0.000,0.016,0.021,0.000,0.019,0.000,0.000,...,0.037,0.018,0.000,0.000,0.000,0.000,0.015,0.000,0.000,0.000
4,4,0.000,0.009,0.018,0.000,0.023,0.000,0.043,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.010,0.000,0.000,0.000
5,5,0.005,0.009,0.008,0.029,0.021,0.023,0.000,0.006,0.016,...,0.008,0.000,0.008,0.015,0.005,0.000,0.195,0.000,0.006,0.000
6,6,0.011,0.036,0.000,0.015,0.013,0.014,0.027,0.016,0.000,...,0.000,0.000,0.000,0.000,0.160,0.000,0.025,0.053,0.030,0.012
7,7,0.000,0.016,0.023,0.000,0.007,0.008,0.000,0.037,0.004,...,0.022,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.041,0.000
8,8,0.027,0.000,0.000,0.021,0.000,0.000,0.036,0.017,0.000,...,0.000,0.019,0.000,0.000,0.015,0.505,0.000,0.018,0.000,0.000
9,9,0.019,0.000,0.000,0.037,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.019,0.000,0.000,0.000,0.000,0.000


In [26]:
# View the ten most common terms in each cluster.
cluster0_train_tfidf = X_train_tfidf[X_train_tfidf['clusters'] == 0]
cluster0_train_tfidf.mean().sort_values(ascending=False)[:10]

day            0.028
freedom        0.022
god            0.021
right          0.020
union          0.019
constitution   0.019
america        0.019
make           0.019
man            0.018
old            0.017
dtype: float64

In [27]:
cluster0_eval_tfidf = X_eval_tfidf[X_eval_tfidf['clusters'] == 0]
cluster0_eval_tfidf.mean().sort_values(ascending=False)[:10]

principle    0.031
law          0.029
believe      0.027
faith        0.026
states       0.024
way          0.023
new          0.023
generation   0.021
union        0.021
case         0.021
dtype: float64

In [28]:
cluster1_train_tfidf = X_train_tfidf[X_train_tfidf['clusters'] == 1]
cluster1_train_tfidf.mean().sort_values(ascending=False)[1:11]

time         0.264
nation       0.228
change       0.047
great        0.040
government   0.035
live         0.032
make         0.032
require      0.031
god          0.029
believe      0.029
dtype: float64

In [29]:
cluster1_eval_tfidf = X_eval_tfidf[X_eval_tfidf['clusters'] == 1]
cluster1_eval_tfidf.mean().sort_values(ascending=False)[1:11]

nation    0.289
time      0.209
freedom   0.134
great     0.101
new       0.095
right     0.094
think     0.075
speak     0.073
power     0.062
future    0.060
dtype: float64

In [30]:
cluster2_train_tfidf = X_train_tfidf[X_train_tfidf['clusters'] == 2]
cluster2_train_tfidf.mean().sort_values(ascending=False)[1:11]

good         0.444
make         0.088
power        0.070
government   0.061
hope         0.060
life         0.043
discipline   0.043
equal        0.040
public       0.036
peace        0.036
dtype: float64

In [31]:
cluster2_eval_tfidf = X_eval_tfidf[X_eval_tfidf['clusters'] == 2]
cluster2_eval_tfidf.mean().sort_values(ascending=False)[1:11]

good       0.357
land       0.191
national   0.175
action     0.141
place      0.138
hold       0.131
friend     0.128
meet       0.126
right      0.116
use        0.101
dtype: float64

In [33]:
cluster3_train_tfidf = X_train_tfidf[
    X_train_tfidf['clusters'] == 3]
cluster3_train_tfidf.mean().sort_values(ascending=False)[1:11]

life      0.323
hand      0.219
heart     0.097
new       0.067
present   0.057
faith     0.052
know      0.043
great     0.041
way       0.037
make      0.030
dtype: float64

In [35]:
cluster3_eval_tfidf = X_eval_tfidf[
    X_eval_tfidf['clusters'] == 3]
cluster3_eval_tfidf.mean().sort_values(ascending=False)[1:11]

life             0.279
hand             0.260
national         0.191
value            0.134
form             0.105
administration   0.096
leader           0.088
majority         0.086
heart            0.080
day              0.075
dtype: float64

In [36]:
cluster4_train_tfidf = X_train_tfidf[
    X_train_tfidf['clusters'] == 4]
cluster4_train_tfidf.mean().sort_values(ascending=False)[1:11]

citizen          0.306
fellow           0.276
fellow citizen   0.175
president        0.095
country          0.064
man              0.054
great            0.049
free             0.048
duty             0.044
americans        0.043
dtype: float64

In [37]:
cluster4_eval_tfidf = X_eval_tfidf[
    X_eval_tfidf['clusters'] == 4]
cluster4_eval_tfidf.mean().sort_values(ascending=False)[1:11]

citizen          0.458
ask              0.247
fellow citizen   0.203
fellow           0.181
america          0.154
today            0.146
hope             0.143
world            0.125
mind             0.120
heart            0.111
dtype: float64

In [38]:
cluster5_train_tfidf = X_train_tfidf[
    X_train_tfidf['clusters'] == 5]
cluster5_train_tfidf.mean().sort_values(ascending=False)[1:11]

world           0.195
people          0.194
free            0.054
states          0.049
united          0.047
great           0.039
shall           0.037
united states   0.036
economic        0.035
peace           0.030
dtype: float64

In [39]:
cluster5_eval_tfidf = X_eval_tfidf[
    X_eval_tfidf['clusters'] == 5]
cluster5_eval_tfidf.mean().sort_values(ascending=False)[1:11]

people            0.177
world             0.175
peace             0.097
america           0.095
great             0.086
american people   0.075
purpose           0.069
american          0.068
year              0.058
americans         0.058
dtype: float64

In [40]:
cluster6_train_tfidf = X_train_tfidf[
    X_train_tfidf['clusters'] == 6]
cluster6_train_tfidf.mean().sort_values(ascending=False)[1:11]

need           0.257
word           0.160
deny           0.098
write          0.053
constitution   0.043
speak          0.040
factory        0.040
great          0.039
material       0.038
moment         0.037
dtype: float64

In [41]:
cluster6_eval_tfidf = X_eval_tfidf[
    X_eval_tfidf['clusters'] == 6]
cluster6_eval_tfidf.mean().sort_values(ascending=False)[1:11]

need        0.501
action      0.152
serve       0.126
care        0.119
congress    0.109
executive   0.107
face        0.106
country     0.100
authority   0.097
today       0.097
dtype: float64

In [42]:
cluster7_train_tfidf = X_train_tfidf[
    X_train_tfidf['clusters'] == 7]
cluster7_train_tfidf.mean().sort_values(ascending=False)[1:11]

government   0.310
support      0.135
shall        0.098
people       0.065
federal      0.057
year         0.041
right        0.041
power        0.039
law          0.039
states       0.038
dtype: float64

In [43]:
cluster7_eval_tfidf = X_eval_tfidf[
    X_eval_tfidf['clusters'] == 7]
cluster7_eval_tfidf.mean().sort_values(ascending=False)[1:11]

government      0.356
states          0.089
opportunity     0.076
continue        0.076
support         0.069
united states   0.067
let             0.063
united          0.061
freedom         0.059
union           0.059
dtype: float64

In [44]:
cluster8_train_tfidf = X_train_tfidf[
    X_train_tfidf['clusters'] == 8]
cluster8_train_tfidf.mean().sort_values(ascending=False)[1:11]

work        0.505
sacrifice   0.063
let         0.056
good        0.051
return      0.049
god         0.046
thing       0.046
moral       0.043
seek        0.042
right       0.039
dtype: float64

In [45]:
cluster8_eval_tfidf = X_eval_tfidf[
    X_eval_tfidf['clusters'] == 8]
cluster8_eval_tfidf.mean().sort_values(ascending=False)[1:11]

work        0.638
know        0.093
new         0.089
people      0.080
task        0.080
old         0.075
friend      0.074
man woman   0.062
woman       0.061
stand       0.060
dtype: float64

In [46]:
cluster9_train_tfidf = X_train_tfidf[
    X_train_tfidf['clusters'] == 9]
cluster9_train_tfidf.mean().sort_values(ascending=False)[1:11]

let          0.506
begin        0.133
fear         0.081
renew        0.058
remember     0.056
hope         0.047
hard         0.041
understand   0.040
america      0.037
far          0.034
dtype: float64

In [47]:
cluster9_eval_tfidf = X_eval_tfidf[
    X_eval_tfidf['clusters'] == 9]
cluster9_eval_tfidf.mean().sort_values(ascending=False)[1:11]

let              0.554
seek             0.197
people           0.159
child            0.140
responsibility   0.138
community        0.135
country          0.120
turn             0.070
end              0.068
generation       0.067
dtype: float64

## LSA

In [71]:
# Re-run KMeans and extract cluster information.
model_lsa = KMeans(n_clusters=10, random_state=42).fit(X_train_lsa_scaled)

# Extract cluster assignments for each data point.
labels = model_lsa.labels_

In [72]:
# Create cluster assignment for eval, holdout groups.
X_eval_lsa_labels = model_lsa.predict(X_eval_lsa_scaled)
X_holdout_lsa_labels = model_lsa.predict(X_holdout_lsa_scaled)

# Create a column for cluster labels.
X_eval_lsa_scaled['clusters'] = X_eval_lsa_labels
X_holdout_lsa_scaled['clusters'] = X_holdout_lsa_labels

X_train_lsa_scaled['clusters'] = labels

In [73]:
# Aggregate by cluster.
X_train_lsa_clusters = X_train_lsa_scaled.groupby(
    ['clusters'], as_index=False).mean()
X_train_lsa_clusters

,clusters,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,0,0.187,0.375,0.525,0.373,0.489,0.391,0.470,0.483,0.498,...,0.442,0.508,0.542,0.506,0.487,0.468,0.506,0.413,0.599,0.341
1,1,0.482,0.430,0.451,0.384,0.441,0.516,0.437,0.448,0.222,...,0.451,0.450,0.528,0.508,0.468,0.516,0.421,0.370,0.553,0.373
2,2,0.553,0.488,0.426,0.233,0.258,0.349,0.244,0.399,0.470,...,0.430,0.466,0.495,0.427,0.480,0.507,0.475,0.373,0.554,0.412
3,3,0.391,0.350,0.465,0.308,0.470,0.474,0.390,0.557,0.500,...,0.420,0.440,0.491,0.474,0.478,0.562,0.473,0.357,0.591,0.429
4,4,0.336,0.360,0.517,0.342,0.475,0.446,0.424,0.465,0.498,...,0.445,0.470,0.458,0.455,0.518,0.528,0.463,0.359,0.521,0.404
5,5,0.202,0.364,0.514,0.358,0.465,0.416,0.442,0.485,0.452,...,0.452,0.471,0.461,0.471,0.487,0.523,0.467,0.388,0.585,0.400
6,6,0.436,0.349,0.447,0.377,0.370,0.444,0.559,0.487,0.666,...,0.445,0.460,0.471,0.448,0.463,0.526,0.460,0.371,0.580,0.391
7,7,0.609,0.439,0.452,0.351,0.390,0.373,0.401,0.482,0.237,...,0.453,0.457,0.469,0.452,0.468,0.522,0.472,0.396,0.565,0.405
8,8,0.594,0.330,0.514,0.434,0.452,0.329,0.570,0.636,0.322,...,0.458,0.477,0.480,0.456,0.490,0.518,0.476,0.376,0.572,0.375
9,9,0.632,0.394,0.402,0.431,0.637,0.339,0.474,0.510,0.361,...,0.456,0.459,0.457,0.441,0.491,0.526,0.457,0.359,0.580,0.397


In [74]:
# View the ten most common terms in each cluster.
cluster0_train_lsa = X_train_lsa_scaled[X_train_lsa_scaled['clusters'] == 0]
cluster0_train_lsa.mean().sort_values(ascending=False)[:10]

29   0.851
26   0.788
32   0.738
41   0.720
24   0.713
44   0.711
31   0.674
25   0.672
36   0.655
50   0.639
dtype: float64

In [75]:
cluster0_eval_lsa = X_eval_lsa_scaled[X_eval_lsa_scaled['clusters'] == 0]
cluster0_eval_lsa.mean().sort_values(ascending=False)[:10]

0   nan
1   nan
2   nan
3   nan
4   nan
5   nan
6   nan
7   nan
8   nan
9   nan
dtype: float64

It appears that cluster0 is a cluster that deals primarily with **people** of one sort or another. Common words in the top ten for each include "man", "woman", "man woman", and "say". 

In [76]:
cluster1_train_lsa = X_train_lsa_scaled[X_train_lsa_scaled['clusters'] == 1]
cluster1_train_lsa.mean().sort_values(ascending=False)[1:11]

39   0.850
60   0.703
53   0.679
52   0.663
42   0.656
37   0.653
32   0.641
47   0.628
41   0.607
76   0.605
dtype: float64

In [77]:
cluster1_eval_lsa = X_eval_lsa_scaled[X_eval_lsa_scaled['clusters'] == 1]
cluster1_eval_lsa.mean().sort_values(ascending=False)[1:11]

52   0.750
57   0.744
39   0.740
9    0.738
88   0.716
0    0.711
60   0.708
76   0.674
83   0.636
47   0.620
dtype: float64

Cluster1 has only one word in common for the top ten of both groups: "today". However, it is apparent that this cluster has a lot to do with **ideals**, with words like "good"/"great", "world", "peace", "principle", "nation", "state", and "constitution".

In [78]:
cluster2_train_lsa = X_train_lsa_scaled[X_train_lsa_scaled['clusters'] == 2]
cluster2_train_lsa.mean().sort_values(ascending=False)[1:11]

9    0.759
76   0.569
98   0.554
0    0.553
87   0.549
36   0.543
69   0.539
41   0.528
31   0.514
50   0.514
dtype: float64

In [79]:
cluster2_eval_lsa = X_eval_lsa_scaled[X_eval_lsa_scaled['clusters'] == 2]
cluster2_eval_lsa.mean().sort_values(ascending=False)[1:11]

9    0.782
32   0.623
69   0.614
27   0.608
98   0.595
64   0.585
1    0.563
10   0.550
41   0.541
87   0.534
dtype: float64

Cluster2 has a less obvious theme that I would like to call **values**. Two common words are "hand" and "material", but other important words include "strong", "life", "power", "liberty", "trust", and "value".

In [80]:
cluster3_train_lsa = X_train_lsa_scaled[X_train_lsa_scaled['clusters'] == 3]
cluster3_train_lsa.mean().sort_values(ascending=False)[1:11]

16   0.711
76   0.593
98   0.591
95   0.562
85   0.560
33   0.559
7    0.557
41   0.555
18   0.553
58   0.550
dtype: float64

In [81]:
cluster3_eval_lsa = X_eval_lsa_scaled[X_eval_lsa_scaled['clusters'] == 3]
cluster3_eval_lsa.mean().sort_values(ascending=False)[1:11]

95   0.634
98   0.625
76   0.614
16   0.607
50   0.601
67   0.591
85   0.573
93   0.565
7    0.562
60   0.561
dtype: float64

In [82]:
cluster4_train_lsa = X_train_lsa_scaled[X_train_lsa_scaled['clusters'] == 4]
cluster4_train_lsa.mean().sort_values(ascending=False)[1:11]

81   0.558
58   0.558
29   0.552
61   0.547
31   0.546
38   0.533
95   0.528
87   0.521
98   0.521
94   0.518
dtype: float64

In [83]:
cluster4_eval_lsa = X_eval_lsa_scaled[X_eval_lsa_scaled['clusters'] == 4]
cluster4_eval_lsa.mean().sort_values(ascending=False)[1:11]

38   0.610
22   0.562
31   0.554
80   0.542
81   0.541
95   0.540
69   0.538
60   0.538
98   0.537
29   0.537
dtype: float64

In [84]:
cluster5_train_lsa = X_train_lsa_scaled[X_train_lsa_scaled['clusters'] == 5]
cluster5_train_lsa.mean().sort_values(ascending=False)[1:11]

98   0.585
69   0.542
85   0.540
87   0.539
11   0.530
95   0.523
9    0.521
44   0.520
64   0.517
58   0.516
dtype: float64

In [85]:
cluster5_eval_lsa = X_eval_lsa_scaled[X_eval_lsa_scaled['clusters'] == 5]
cluster5_eval_lsa.mean().sort_values(ascending=False)[1:11]

98   0.602
69   0.550
85   0.538
47   0.527
64   0.526
36   0.522
11   0.520
60   0.519
58   0.519
44   0.519
dtype: float64

In [86]:
cluster6_train_lsa = X_train_lsa_scaled[X_train_lsa_scaled['clusters'] == 6]
cluster6_train_lsa.mean().sort_values(ascending=False)[1:11]

12   0.667
8    0.666
98   0.580
6    0.559
85   0.557
17   0.550
58   0.536
76   0.536
87   0.535
41   0.534
dtype: float64

In [87]:
cluster6_eval_lsa = X_eval_lsa_scaled[X_eval_lsa_scaled['clusters'] == 6]
cluster6_eval_lsa.mean().sort_values(ascending=False)[1:11]

17   0.617
87   0.612
12   0.607
98   0.586
8    0.582
18   0.581
76   0.579
11   0.566
64   0.559
16   0.557
dtype: float64

In [88]:
cluster7_train_lsa = X_train_lsa_scaled[X_train_lsa_scaled['clusters'] == 7]
cluster7_train_lsa.mean().sort_values(ascending=False)[1:11]

0    0.609
11   0.597
98   0.565
85   0.546
69   0.544
33   0.527
58   0.525
95   0.522
76   0.521
87   0.520
dtype: float64

In [89]:
cluster7_eval_lsa = X_eval_lsa_scaled[X_eval_lsa_scaled['clusters'] == 7]
cluster7_eval_lsa.mean().sort_values(ascending=False)[1:11]

0    0.660
11   0.612
69   0.601
98   0.573
88   0.563
67   0.561
95   0.547
87   0.535
54   0.528
51   0.523
dtype: float64

In [90]:
cluster8_train_lsa = X_train_lsa_scaled[X_train_lsa_scaled['clusters'] == 8]
cluster8_train_lsa.mean().sort_values(ascending=False)[1:11]

7    0.636
0    0.594
41   0.589
14   0.574
98   0.572
6    0.570
19   0.566
33   0.543
76   0.539
85   0.536
dtype: float64

In [91]:
cluster8_eval_lsa = X_eval_lsa_scaled[X_eval_lsa_scaled['clusters'] == 8]
cluster8_eval_lsa.mean().sort_values(ascending=False)[1:11]

98   0.600
19   0.596
7    0.595
0    0.583
14   0.583
55   0.581
6    0.572
49   0.571
41   0.564
64   0.562
dtype: float64

In [92]:
cluster9_train_lsa = X_train_lsa_scaled[X_train_lsa_scaled['clusters'] == 9]
cluster9_train_lsa.mean().sort_values(ascending=False)[1:11]

9    0.691
4    0.637
0    0.632
98   0.580
12   0.578
87   0.541
36   0.539
85   0.539
95   0.526
58   0.521
dtype: float64

In [94]:
cluster9_eval_lsa = X_eval_lsa_scaled[X_eval_lsa_scaled['clusters'] == 9]
cluster9_eval_lsa.mean().sort_values(ascending=False)[1:11]

9    0.659
12   0.640
4    0.617
0    0.588
85   0.580
6    0.559
10   0.556
36   0.556
76   0.554
55   0.552
dtype: float64

Lastly, cluster3 is what I will call **now**. Common words include "time" and "government", and other relevant words include "change", "history", "future", "courage", "generation", and "god" (lemmatized, so we'll say "God").

## Latent Dirichlet Allocation (LDA)

### Set up text for LDA

In [95]:
# Removing numerals.
sent_df['sentence_tokens'] = sent_df.sentence.map(
    lambda x: re.sub(r'\d+', '', x))
# Lower case.
sent_df['sentence_tokens'] = sent_df.sentence_tokens.map(lambda x: x.lower())
print(sent_df['sentence_tokens'][0][:500])

0    fellow - citizens of the senate and of the hou...
0    friend and fellow citizens : call upon to unde...
0             fellow - citizens of the united states :
0     be certain that  fellow americans expect that...
0     friend , before  begin the expression of thos...
0    vice president johnson , mr. speaker , mr. chi...
0    senator hatfield , mr. chief justice , mr. pre...
0    mr. chief justice , mr. president , vice presi...
0     fellow citizen , today  celebrate the mystery...
0     fellow citizen :  stand here today humble by ...
Name: sentence_tokens, dtype: object


In [96]:
# Tokenize.
sent_df['sentence_tokens'] = sent_df.sentence_tokens.map(
    lambda x: RegexpTokenizer(r'\w+').tokenize(x))
print(sent_df['sentence_tokens'][0][:25])

0    [fellow, citizens, of, the, senate, and, of, t...
0    [friend, and, fellow, citizens, call, upon, to...
0          [fellow, citizens, of, the, united, states]
0    [be, certain, that, fellow, americans, expect,...
0    [friend, before, begin, the, expression, of, t...
0    [vice, president, johnson, mr, speaker, mr, ch...
0    [senator, hatfield, mr, chief, justice, mr, pr...
0    [mr, chief, justice, mr, president, vice, pres...
0    [fellow, citizen, today, celebrate, the, myste...
0    [fellow, citizen, stand, here, today, humble, ...
Name: sentence_tokens, dtype: object


In [97]:
# Stemming.
snowball = SnowballStemmer("english")
sent_df['sentence_tokens'] = sent_df.sentence_tokens.map(
    lambda x: [snowball.stem(token) for token in x])
print(sent_df['sentence_tokens'][0][:25])

0    [fellow, citizen, of, the, senat, and, of, the...
0    [friend, and, fellow, citizen, call, upon, to,...
0              [fellow, citizen, of, the, unit, state]
0    [be, certain, that, fellow, american, expect, ...
0    [friend, befor, begin, the, express, of, those...
0    [vice, presid, johnson, mr, speaker, mr, chief...
0    [senat, hatfield, mr, chief, justic, mr, presi...
0    [mr, chief, justic, mr, presid, vice, presid, ...
0    [fellow, citizen, today, celebr, the, mysteri,...
0    [fellow, citizen, stand, here, today, humbl, b...
Name: sentence_tokens, dtype: object


In [98]:
# Stop words.
stop_en = stopwords.words('english')
sent_df['sentence_tokens'] = sent_df.sentence_tokens.map(
    lambda x: [t for t in x if t not in stop_en])
print(sent_df['sentence_tokens'][0][:25])

0               [fellow, citizen, senat, hous, repres]
0    [friend, fellow, citizen, call, upon, undertak...
0                       [fellow, citizen, unit, state]
0    [certain, fellow, american, expect, induct, pr...
0    [friend, befor, begin, express, thought, deem,...
0    [vice, presid, johnson, mr, speaker, mr, chief...
0    [senat, hatfield, mr, chief, justic, mr, presi...
0    [mr, chief, justic, mr, presid, vice, presid, ...
0    [fellow, citizen, today, celebr, mysteri, amer...
0    [fellow, citizen, stand, today, humbl, task, b...
Name: sentence_tokens, dtype: object


In [99]:
# Final cleaning.
sent_df['sentence_tokens'] = sent_df.sentence_tokens.map(
    lambda x: [t for t in x if len(t) > 1])
print(sent_df['sentence_tokens'][0][:25])

0               [fellow, citizen, senat, hous, repres]
0    [friend, fellow, citizen, call, upon, undertak...
0                       [fellow, citizen, unit, state]
0    [certain, fellow, american, expect, induct, pr...
0    [friend, befor, begin, express, thought, deem,...
0    [vice, presid, johnson, mr, speaker, mr, chief...
0    [senat, hatfield, mr, chief, justic, mr, presi...
0    [mr, chief, justic, mr, presid, vice, presid, ...
0    [fellow, citizen, today, celebr, mysteri, amer...
0    [fellow, citizen, stand, today, humbl, task, b...
Name: sentence_tokens, dtype: object


### Run LDA

In [100]:
texts = sent_df['sentence_tokens'].values
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda = LdaModel(corpus,
               id2word=dictionary,
               num_topics=10,
               passes=5,
               minimum_probability=0,
               random_state=15)

In [101]:
# Print topics
lda.print_topics()

[(0,
  '0.013*"america" + 0.012*"nation" + 0.011*"state" + 0.010*"shall" + 0.010*"govern" + 0.009*"god" + 0.008*"let" + 0.008*"world" + 0.007*"renew" + 0.007*"must"'),
 (1,
  '0.010*"great" + 0.010*"nation" + 0.009*"good" + 0.009*"everi" + 0.008*"peopl" + 0.007*"citizen" + 0.007*"mean" + 0.006*"may" + 0.006*"man" + 0.006*"free"'),
 (2,
  '0.016*"may" + 0.013*"constitut" + 0.009*"take" + 0.008*"state" + 0.008*"law" + 0.006*"great" + 0.006*"shall" + 0.006*"say" + 0.006*"union" + 0.006*"first"'),
 (3,
  '0.011*"american" + 0.010*"right" + 0.009*"ani" + 0.009*"onli" + 0.009*"state" + 0.008*"must" + 0.008*"govern" + 0.007*"generat" + 0.007*"peopl" + 0.006*"old"'),
 (4,
  '0.014*"peopl" + 0.012*"govern" + 0.011*"life" + 0.009*"countri" + 0.008*"strength" + 0.007*"ask" + 0.007*"free" + 0.007*"make" + 0.006*"good" + 0.005*"man"'),
 (5,
  '0.013*"peopl" + 0.011*"work" + 0.010*"new" + 0.009*"nation" + 0.008*"make" + 0.008*"great" + 0.007*"american" + 0.007*"man" + 0.007*"freedom" + 0.006*"today"

In [102]:
# Refactoring results of LDA into numpy matrix.
hm = np.array([[y for (x,y) in lda[corpus[i]]] for i in range(len(corpus))])

In [103]:
# Reduce dimensionality using t-SNE.
tsne = TSNE(random_state=15, perplexity=30, early_exaggeration=120)
embedding = tsne.fit_transform(hm)
embedding = pd.DataFrame(embedding, columns=['x','y'])
embedding['hue'] = hm.argmax(axis=1)

In [104]:
# Scatter plot using Bokeh.
source = ColumnDataSource(
    data=dict(x=embedding.x,
              y=embedding.y,
              colors=[all_palettes['Set1'][8] for i in embedding.hue],
              sentence=sent_df.sentence,
              President=sent_df.President,
              alpha=[0.9] * embedding.shape[0],
              size=[7] * embedding.shape[0]
              )
)
hover_tsne = HoverTool(names=["sent_df"], tooltips="""
    <div style="margin: 10">
        <div style="margin: 0 auto; width:300px;">
            <span style="font-size: 12px; font-weight: bold;">Title:</span>
            <span style="font-size: 12px">@title</span>
            <span style="font-size: 12px; font-weight: bold;">Year:</span>
            <span style="font-size: 12px">@year</span>
        </div>
    </div>
    """)
tools_tsne = [hover_tsne, 'pan', 'wheel_zoom', 'reset']
plot_tsne = figure(plot_width=700, plot_height=700,
                   tools=tools_tsne, title='Inaugural Addresses')
plot_tsne.circle('x', 'y', size='size', fill_color='colors',alpha='alpha',
                 line_alpha=0, line_width=0.01, source=source, name="sent_df")

callback = CustomJS(args=dict(source=source), code=
    """var data = source.data;
    var f = cb_obj.value
    x = data['x']
    y = data['y']
    colors = data['colors']
    alpha = data['alpha']
    title = data['title']
    President = data['President']
    size = data['size']
    for (i = 0; i < x.length; i++) {
        if (year[i] <= f) {
            alpha[i] = 0.9
            size[i] = 7
        } else {
            alpha[i] = 0.05
            size[i] = 4
        }
    }
    source.change.emit();
    """)

slider = Slider(
    start=sent_df.President.min(), end=sent_df.sentence.max(),values=
    ['Washington', 'Jefferson', 'Lincoln', 'FDR', 'Eisenhower', 'Kennedy',
     'Reagan', 'GHWBush', 'Clinton', 'Obama'], step=1, title="Inaugural Speeches")
slider.js_on_change('value', callback)

layout = column(plot_tsne)
show(layout)

ValueError: expected a value of type Real, got clinton of type str

# Prepare for Predictive Modeling

In [105]:
'''Create baseline score to beat. Bush41 had the most sentences, so guessing 
him for all sentences would give this percentage.
'''

print('Baseline score to beat:', sum(
    (sent_df.President == 'ghwbush') / len(sent_df.President)))

Baseline score to beat: 0.15491452991452967


In [106]:
# Pipeline helpers.
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=15)

In [109]:
# Instantiate the models.
log_reg = LogisticRegression(solver='lbfgs', max_iter=1000, random_state=15)
tree = DecisionTreeClassifier(random_state=15)
forest = RandomForestClassifier(max_depth=10, random_state=15)
boost = GradientBoostingClassifier(random_state=15)
nb = BernoulliNB()

In [111]:
# Set up _kwargs files for convenience.
tfidf_kwargs = {'X_train': X_train_tfidf,'y_train': y_train,
                'X_eval': X_eval_tfidf,'y_eval': y_eval}
                #'X_holdout': X_holdout_tfidf, 'y_holdout': y_holdout}

lsa_kwargs = {'X_train': X_train_lsa_scaled, 'y_train': y_train,
              'X_eval': X_eval_lsa_scaled, 'y_eval': y_eval}
              #'X_holdout': X_holdout_tfidf_scaled, 'y_holdout': y_holdout}

In [112]:
# Tune parameter grids.
log_reg_params = {'model__C': [1, 10, 100, 1000]}
tree_params = {'model__criterion': ['gini']}
forest_params = {'model__n_estimators': [100, 200, 300,400],
                 'model__max_depth': [None, 5, 10]}
boost_params = {'model__n_estimators': [100]}
nb_params = {'model__alpha': [1]}

In [113]:
# Function to fit and predict all working kernals.


def fit_and_predict(model, params: Dict,
                    X_train: pd.DataFrame,
                    y_train: pd.DataFrame,
                    X_eval: pd.DataFrame,
                    y_eval: pd.DataFrame) -> None:
    """
    Takes an instantiated sklearn model, training data (X_train, y_train), 
    and performs cross-validation and then prints the mean of the cross-
    validation accuracies.
    """
    assert len(X_train) == len(y_train)
    assert len(X_eval) == len(y_eval)
    # assert len(X_holdout) == len(y_holdout)
    pipe = Pipeline(steps=[('model', model)])
    clf = GridSearchCV(pipe, cv=skf, param_grid=params, n_jobs=2)
    clf.fit(X_train, y_train)
    print('The mean cross_val accuracy on train is',
          f'{clf.cv_results_["mean_test_score"]}.')
    print('The std of the cross_val accuracy is',
          f'{clf.cv_results_["std_test_score"]}.')
    y_pred = clf.predict(X_eval)
    print(classification_report(y_eval, y_pred))
    print(confusion_matrix(y_eval, y_pred))

## Logistic Regression

### Tfidf

In [114]:
fit_and_predict(log_reg, params=log_reg_params, **tfidf_kwargs)

The mean cross_val accuracy on train is [0.34410646 0.34220532 0.31749049 0.28136882].
The std of the cross_val accuracy is [0.0365393  0.01761933 0.0400676  0.05275828].
              precision    recall  f1-score   support

     clinton       0.33      0.12      0.18        16
  eisenhower       0.35      0.42      0.38        19
         fdr       0.25      0.06      0.10        16
     ghwbush       0.15      0.44      0.22        25
   jefferson       1.00      0.10      0.18        10
     kennedy       0.00      0.00      0.00         9
     lincoln       0.58      0.62      0.60        29
       obama       0.00      0.00      0.00        25
      reagan       0.24      0.33      0.28        24
  washington       0.00      0.00      0.00         3

   micro avg       0.28      0.28      0.28       176
   macro avg       0.29      0.21      0.19       176
weighted avg       0.30      0.28      0.25       176

[[ 2  1  0  9  0  1  0  0  3  0]
 [ 0  8  0  7  0  0  1  2  1  0]
 [ 0

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### LSA (Latent Semantic Analysis)

In [115]:
fit_and_predict(log_reg, params=log_reg_params, **lsa_kwargs)

The mean cross_val accuracy on train is [0.34030418 0.3365019  0.30608365 0.2851711 ].
The std of the cross_val accuracy is [0.02437226 0.045907   0.02789979 0.05238789].
              precision    recall  f1-score   support

     clinton       0.23      0.19      0.21        16
  eisenhower       0.27      0.47      0.35        19
         fdr       0.36      0.25      0.30        16
     ghwbush       0.16      0.36      0.22        25
   jefferson       0.33      0.10      0.15        10
     kennedy       0.50      0.11      0.18         9
     lincoln       0.58      0.52      0.55        29
       obama       0.25      0.08      0.12        25
      reagan       0.26      0.25      0.26        24
  washington       0.00      0.00      0.00         3

   micro avg       0.28      0.28      0.28       176
   macro avg       0.29      0.23      0.23       176
weighted avg       0.32      0.28      0.27       176

[[ 3  2  1  8  0  1  0  0  1  0]
 [ 0  9  0  7  0  0  2  1  0  0]
 [ 0

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


## Decision Trees

### Tfidf

In [116]:
fit_and_predict(tree, params=tree_params, **tfidf_kwargs)

The mean cross_val accuracy on train is [0.26996198].
The std of the cross_val accuracy is [0.0617106].
              precision    recall  f1-score   support

     clinton       0.12      0.12      0.12        16
  eisenhower       0.21      0.26      0.23        19
         fdr       0.08      0.06      0.07        16
     ghwbush       0.11      0.28      0.16        25
   jefferson       0.00      0.00      0.00        10
     kennedy       0.40      0.22      0.29         9
     lincoln       0.55      0.41      0.47        29
       obama       0.18      0.08      0.11        25
      reagan       0.33      0.25      0.29        24
  washington       0.00      0.00      0.00         3

   micro avg       0.21      0.21      0.21       176
   macro avg       0.20      0.17      0.17       176
weighted avg       0.24      0.21      0.21       176

[[ 2  1  0  9  0  0  1  1  2  0]
 [ 1  5  2  9  0  0  1  0  1  0]
 [ 1  3  1  7  1  0  0  1  2  0]
 [ 4  4  2  7  0  0  3  2  2  1]
 [ 0 

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


### LSA

In [117]:
fit_and_predict(tree, params=tree_params, **lsa_kwargs)

The mean cross_val accuracy on train is [0.19961977].
The std of the cross_val accuracy is [0.04623214].
              precision    recall  f1-score   support

     clinton       0.00      0.00      0.00        16
  eisenhower       0.24      0.32      0.27        19
         fdr       0.23      0.19      0.21        16
     ghwbush       0.12      0.20      0.15        25
   jefferson       0.11      0.10      0.11        10
     kennedy       0.00      0.00      0.00         9
     lincoln       0.58      0.38      0.46        29
       obama       0.14      0.12      0.13        25
      reagan       0.13      0.12      0.13        24
  washington       0.00      0.00      0.00         3

   micro avg       0.18      0.18      0.18       176
   macro avg       0.15      0.14      0.15       176
weighted avg       0.20      0.18      0.19       176

[[ 0  4  1  5  0  1  0  4  1  0]
 [ 0  6  0  6  1  0  1  3  1  1]
 [ 1  3  3  3  2  0  1  1  2  0]
 [ 2  1  1  5  2  1  4  5  4  0]
 [ 1

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


## Random Forest

### Tfidf

In [118]:
fit_and_predict(forest, params=forest_params, **tfidf_kwargs)

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


The mean cross_val accuracy on train is [0.30798479 0.31749049 0.32509506 0.32889734 0.28326996 0.29277567
 0.28707224 0.28326996 0.28707224 0.29657795 0.30228137 0.30228137].
The std of the cross_val accuracy is [0.0383545  0.03435724 0.03282714 0.03598003 0.05548412 0.05193235
 0.04677472 0.04852622 0.04649525 0.04687379 0.04612755 0.04741648].
              precision    recall  f1-score   support

     clinton       0.27      0.25      0.26        16
  eisenhower       0.21      0.32      0.25        19
         fdr       0.12      0.06      0.08        16
     ghwbush       0.15      0.32      0.21        25
   jefferson       0.50      0.30      0.37        10
     kennedy       1.00      0.11      0.20         9
     lincoln       0.56      0.52      0.54        29
       obama       0.07      0.04      0.05        25
      reagan       0.27      0.25      0.26        24
  washington       0.00      0.00      0.00         3

   micro avg       0.26      0.26      0.26       176
 

### LSA

In [119]:
fit_and_predict(forest, params=forest_params, **lsa_kwargs)

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


The mean cross_val accuracy on train is [0.33460076 0.33460076 0.33460076 0.33269962 0.29277567 0.28326996
 0.30608365 0.30988593 0.30798479 0.32889734 0.33079848 0.33269962].
The std of the cross_val accuracy is [0.04916592 0.05175044 0.0485816  0.03278952 0.06021124 0.04203034
 0.04743791 0.04994803 0.04978877 0.03431607 0.03067429 0.03364358].
              precision    recall  f1-score   support

     clinton       0.43      0.19      0.26        16
  eisenhower       0.24      0.47      0.32        19
         fdr       0.67      0.25      0.36        16
     ghwbush       0.18      0.44      0.26        25
   jefferson       0.00      0.00      0.00        10
     kennedy       1.00      0.11      0.20         9
     lincoln       0.56      0.69      0.62        29
       obama       0.00      0.00      0.00        25
      reagan       0.30      0.29      0.30        24
  washington       0.00      0.00      0.00         3

   micro avg       0.31      0.31      0.31       176
 

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Gradient Boosting Machines

### Tfidf

In [120]:
fit_and_predict(boost, params=boost_params, **tfidf_kwargs)

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


The mean cross_val accuracy on train is [0.28326996].
The std of the cross_val accuracy is [0.03678817].
              precision    recall  f1-score   support

     clinton       0.33      0.25      0.29        16
  eisenhower       0.19      0.26      0.22        19
         fdr       0.25      0.12      0.17        16
     ghwbush       0.14      0.36      0.20        25
   jefferson       0.00      0.00      0.00        10
     kennedy       0.00      0.00      0.00         9
     lincoln       0.46      0.38      0.42        29
       obama       0.00      0.00      0.00        25
      reagan       0.33      0.33      0.33        24
  washington       0.00      0.00      0.00         3

   micro avg       0.22      0.22      0.22       176
   macro avg       0.17      0.17      0.16       176
weighted avg       0.21      0.22      0.21       176

[[ 4  3  0  6  0  0  1  0  2  0]
 [ 0  5  1  7  0  1  1  3  1  0]
 [ 0  0  2  8  0  0  1  1  4  0]
 [ 1  6  2  9  0  1  2  2  2  0]
 [ 0

### LSA

In [121]:
fit_and_predict(boost, params=boost_params, **lsa_kwargs)

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


The mean cross_val accuracy on train is [0.25665399].
The std of the cross_val accuracy is [0.04243727].
              precision    recall  f1-score   support

     clinton       0.12      0.06      0.08        16
  eisenhower       0.25      0.37      0.30        19
         fdr       0.29      0.12      0.17        16
     ghwbush       0.12      0.28      0.17        25
   jefferson       0.00      0.00      0.00        10
     kennedy       0.25      0.11      0.15         9
     lincoln       0.50      0.59      0.54        29
       obama       0.27      0.12      0.17        25
      reagan       0.31      0.33      0.32        24
  washington       0.00      0.00      0.00         3

   micro avg       0.26      0.26      0.26       176
   macro avg       0.21      0.20      0.19       176
weighted avg       0.26      0.26      0.24       176

[[ 1  2  1 10  0  1  0  1  0  0]
 [ 0  7  0  9  0  0  1  1  1  0]
 [ 0  4  2  4  0  0  4  1  1  0]
 [ 2  3  1  7  0  0  6  0  6  0]
 [ 0

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Naive Bayes

### Tfidf

In [122]:
fit_and_predict(nb, params=nb_params, **tfidf_kwargs)

The mean cross_val accuracy on train is [0.36311787].
The std of the cross_val accuracy is [0.05182334].
              precision    recall  f1-score   support

     clinton       0.00      0.00      0.00        16
  eisenhower       0.50      0.37      0.42        19
         fdr       0.40      0.12      0.19        16
     ghwbush       0.21      0.64      0.31        25
   jefferson       0.00      0.00      0.00        10
     kennedy       0.00      0.00      0.00         9
     lincoln       0.63      0.59      0.61        29
       obama       0.00      0.00      0.00        25
      reagan       0.22      0.42      0.29        24
  washington       1.00      0.33      0.50         3

   micro avg       0.30      0.30      0.30       176
   macro avg       0.30      0.25      0.23       176
weighted avg       0.27      0.30      0.26       176

[[ 0  1  0 11  0  0  0  0  4  0]
 [ 0  7  0  8  0  0  2  0  2  0]
 [ 0  1  2  6  0  0  0  0  7  0]
 [ 0  1  0 16  0  0  1  0  7  0]
 [ 0

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


###  LSA

In [123]:
fit_and_predict(nb, params=nb_params, **lsa_kwargs)

The mean cross_val accuracy on train is [0.16920152].
The std of the cross_val accuracy is [0.01089522].
              precision    recall  f1-score   support

     clinton       0.00      0.00      0.00        16
  eisenhower       0.00      0.00      0.00        19
         fdr       0.00      0.00      0.00        16
     ghwbush       0.14      0.92      0.24        25
   jefferson       0.00      0.00      0.00        10
     kennedy       0.00      0.00      0.00         9
     lincoln       0.60      0.10      0.18        29
       obama       0.00      0.00      0.00        25
      reagan       0.33      0.04      0.07        24
  washington       0.00      0.00      0.00         3

   micro avg       0.15      0.15      0.15       176
   macro avg       0.11      0.11      0.05       176
weighted avg       0.16      0.15      0.07       176

[[ 0  0  0 16  0  0  0  0  0  0]
 [ 0  0  0 18  0  0  1  0  0  0]
 [ 0  0  0 16  0  0  0  0  0  0]
 [ 0  1  0 23  0  0  1  0  0  0]
 [ 0

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Neural Network

## *Note: there are not enough data to effectively run a neural network on this project. Section 5 is merely going through the process for the sake of the capstone.*

In [126]:
# Establish and fit the multi-level perceptron model.
mlp = MLPClassifier(
    hidden_layer_sizes=(3,), random_state=15, max_iter=5000, alpha=0.05)
mlp.fit(X_train_tfidf, y_train)

MLPClassifier(activation='relu', alpha=0.05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(3,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=5000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=15, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [127]:
# Find MLP score.
mlp.score(X_train_tfidf, y_train)

0.7718631178707225

In [128]:
# Find cross-validation score.
cross_val_score(mlp, X_train_tfidf, y_train, cv=5)

array([0.22727273, 0.22222222, 0.24528302, 0.2745098 , 0.24      ])

In [129]:
# Adjust hidden layer parameters.
mlp1 = MLPClassifier(
    hidden_layer_sizes=(5,2,), random_state=15, max_iter=5000, alpha=0.01)
mlp1.fit(X_train_tfidf, y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=5000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=15, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [130]:
# Find accuracy score.
mlp1.score(X_train_tfidf, y_train)

0.8897338403041825

In [131]:
# Cross-validation.
cross_val_score(mlp1, X_train_tfidf, y_train, cv=5)

array([0.20909091, 0.19444444, 0.19811321, 0.25490196, 0.15      ])

In [132]:
# Adjust hidden layer parameters.
mlp2 = MLPClassifier(
    hidden_layer_sizes=(5,2,), random_state=15, max_iter=5000, alpha=0.05)
mlp2.fit(X_train_tfidf, y_train)

MLPClassifier(activation='relu', alpha=0.05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=5000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=15, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [133]:
# Find accuracy score.
mlp2.score(X_train_tfidf, y_train)

0.8878326996197718

In [134]:
# Cross-validation.
cross_val_score(mlp2, X_train_tfidf, y_train, cv=5)

array([0.22727273, 0.19444444, 0.19811321, 0.2745098 , 0.15      ])

In [135]:
#fit_and_predict(log_reg, params=log_reg_params, **bow_kwargs)

In [136]:
#fit_and_predict(tree, params=tree_params, **bow_kwargs)

In [137]:
#fit_and_predict(forest, params=forest_params, **bow_kwargs)

In [138]:
#fit_and_predict(boost, params=boost_params, **bow_kwargs)

In [139]:
#fit_and_predict(nb, params=nb_params, **bow_kwargs)